In [1]:
import pandas as pd
import numpy as np
import cooler
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.patches import Patch
from matplotlib.offsetbox import AnchoredText
import scipy.io
import pyBigWig
import gget
import pysam

import os
import sys
from importlib import reload

import filters
import pore_c_utils as pcu
import binning_utils as binning
import plotting as hicPlot

In [2]:
assembly = pcu.loadAssembly("GRCm39_Assembly.txt")

chromDict = dict(zip(assembly['RefSeq accession'],assembly['Chromosome'].apply(lambda x : f"chr{x}")))

# the reverse for easy lookup
chromDict_r = {v: k for k, v in chromDict.items()}
chromOrder = list(chromDict.values())

In [3]:
dirpath = "/nfs/turbo/umms-indikar/shared/projects/poreC/nagano_2017_rnaseq/"

filepath = f"{dirpath}4DNESZ741PJM_processed_files_2022-05-30-14h-43m.tsv"
summary = pd.read_csv(filepath, sep='\t', comment='#')

summary['Filename'] = summary['File Download URL'].apply(lambda x: x.split("/")[-1])
cols = ['Filename', 'Experiment Accession', 'File Accession', 'Size (MB)', 'File Type', 'File Format',]
summary[cols]

,Filename,Experiment Accession,File Accession,Size (MB),File Type,File Format
0,4DNFI7FO9H19.bam,4DNEXCHOKPKJ,4DNFI7FO9H19,1503.75,read positions,bam
1,4DNFI8CSCJWM.tsv,4DNEXCHOKPKJ,4DNFI8CSCJWM,23.19,isoform expression,tsv
2,4DNFIUW8CG2I.bw,4DNEXCHOKPKJ,4DNFIUW8CG2I,42.86,read counts (plus),bw
3,4DNFIXOTRTRM.bw,4DNEXCHOKPKJ,4DNFIXOTRTRM,42.06,read counts (minus),bw
4,4DNFIYTCHMIZ.tsv,4DNEXCHOKPKJ,4DNFIYTCHMIZ,11.23,gene expression,tsv
5,4DNFI12AUKQS.bw,4DNEXGMSIOHL,4DNFI12AUKQS,31.51,read counts (plus),bw
6,4DNFI3YYNDKI.tsv,4DNEXGMSIOHL,4DNFI3YYNDKI,11.23,gene expression,tsv
7,4DNFIFVPB94O.bw,4DNEXGMSIOHL,4DNFIFVPB94O,30.93,read counts (minus),bw
8,4DNFIPYGE7JR.tsv,4DNEXGMSIOHL,4DNFIPYGE7JR,23.18,isoform expression,tsv
9,4DNFISUXPL3L.bam,4DNEXGMSIOHL,4DNFISUXPL3L,1063.16,read positions,bam


In [24]:
filepath = f"{dirpath}4DNFIYTCHMIZ.tsv"

df = pd.read_csv(filepath, sep='\t')
df.head()

,gene_id,transcript_id(s),length,effective_length,expected_count,TPM,FPKM,posterior_mean_count,posterior_standard_deviation_of_count,pme_TPM,pme_FPKM,TPM_ci_lower_bound,TPM_ci_upper_bound,TPM_coefficient_of_quartile_variation,FPKM_ci_lower_bound,FPKM_ci_upper_bound,FPKM_coefficient_of_quartile_variation
0,10000,10000,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10001,10001,73.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10002,10002,73.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10003,10003,75.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10004,10004,78.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
print(f"{df.shape=}")

df = df[df['TPM'] > 0]
# df = df[df['gene_id'].str.contains("ENSMUSG")]
print(f"{df.shape=}")

df.shape=(81881, 17)
df.shape=(19040, 17)


In [30]:
df['gene_id'].value_counts()

31383                    1
ENSMUSG00000063410.8     1
ENSMUSG00000063383.5     1
ENSMUSG00000063382.6     1
ENSMUSG00000063364.10    1
                        ..
ENSMUSG00000030770.15    1
ENSMUSG00000030769.15    1
ENSMUSG00000030768.12    1
ENSMUSG00000030766.15    1
ENSMUSG00000118386.1     1
Name: gene_id, Length: 19040, dtype: int64

In [18]:
# # fetch data
# geneInfo = gget.info(df['gene_id'], expand=True)

# geneInfo.to_csv("filtered_data/geneNames.csv", index=False)
# print('done.')